You should be able to run this notebook with the test files included in the zipped file. You will need to redefine your out_dir and spectra paths before actually running the fits for yourself. 

In [ ]:
import beat

ModuleNotFoundError: No module named 'beat'

In [ ]:
import numpy as np
import os

The user should define their own function to load their files

Given a path, your function should return "wave", "flux", and "noise"

For example if a file is named "ic5063_150_155.txt" and has 3 columns of data - wave, flux, and noise.

Then use loadtxt or whatever you prefer to read in your data and name your columns accordingly. 

The following load_file works for the files included in the example folder.

In [ ]:
def load_file(filepath):
    """reads file and returns wave, flux, and noise arrays to pass to fitting code"""
    wave, flux, noise = np.loadtxt(filepath, usecols=(0, 1, 2), unpack=True, delimiter=',')
    return row, column, wave, flux, noise

You'll need to define two different dictionaries, named here "target_param" and "fit_instructions".

Target param will include:
- name - your target name
- red - redshift of your object
- minwidth - minimum width that should be attempted to be fit
- maxwidth - maximum width that should be attempted to be fit
- start - the start of your data in array space (I'll change this soon, I swear lol)
- end - the end of your data in array space (and this of course)
- fluxsigma - value to multiply flux sigma by to keep or reject a fit 
- continuum1 - continuum area to the left of your line(s)
- continuum2 - continuum area to the right of your line(s)
- plotmin - x-axis minimum for output plots (wavelength)
- plotmax - x-axis maximum for output plots
- maxcomp - the maximum number of components that should be attempted per line
- lnz - evidence value difference needed to accept a fit with more components. 5.0 is standard, a number larger than 5 will make it harder for a fit with more components to be chosen
- cores - # processes for multiprocessing pool

Fit instructions will include information about the lines you want to fit:
- line1 - this should be your 'main' line that other lines would be locked with, if specified
    - name - name of your line which will be used to name the output file
    - wave - rest wavelength of line
    - minwave - minimum wavelength that should be fit for this one line
    - wave_range - (minwave, minwave + wave_range) will be the allowed wavelengths for the centroid of any component (right?)
    - flux_free - is this line's flux locked in with any other line? For line 1 this will be False
- line2 - this would be a second line that you would like to fit in tandem with line1. most parameters are identical as line1, except:
    - flux_free will be False if this line is locked with another line
    - locked_with - which line is this line locked with ("line1")
    - flux_ratio - what is the flux ratio between the two lines [line1 = (flux_ratio)*line2]
- more lines may be added ('line3', 'line4', etc) - remember to list any lines that will have flux_free = True before any lines that rely on that line 
    
Call the 'Fit' class instance - define an output directory, specify where your data files are, pass your target parameters and fit instructions. 

Finally call mp_handler to start the fits

In [ ]:
def main():
    target_param = {'name': 'ic5063', 'red': 0.01135, 'minwidth': 1.43, 'maxwidth': 33.83,
                    'start': 10, 'end': 450, 'fluxsigma': 3, 'continuum1': (4962, 5000),
                    'continuum2': (5087, 5125), 'plotmin': 4900, 'plotmax': 5150, 'maxcomp': 3, 
                    'lnz': 5.0, 'cores' : 2} 

    """fit_instructions = {
        'line1': {'name': 'oiii', 'wave': 5006.84, 'minwave': 5054, 'wave_range': 20.0, 'flux_free': True},
        'line2': {'name': 'oiii', 'wave': 4958.92, 'flux_free': False, 'locked_with': 'line1', 'flux_ratio': 3}
        }"""
    fit_instructions = {
        'line1': {'name': 'oiii', 'wave': 5006.84, 'minwave': 5054, 'wave_range': 20.0, 'flux_free': True},
        'line2': {'name': 'oiii', 'wave': 4958.92, 'flux_free': False, 'locked_with': 'line1', 'flux_ratio': 3},
        'line3': {'name': 'h-beta', 'wave': 4861.33, 'minwave': 4911, 'wave_range': 15.0, 'flux_free': True}
        }


    fit = beat.Fit(out_dir='ic5063/',
                   spec_dir='spectra',
                   load_file=load_file,
                   fit_instructions=fit_instructions,
                   target_param=target_param
                   )
    fit.mp_handler()

call main() or whatever function you put your dictionaries and fit call

try stopping and restarting the fits

In [ ]:
main()